# Week 4 — Industry Benchmark Suites
### BenchRight LLM Evaluation Master Program (18 Weeks)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:

1. Load benchmark data from Hugging Face datasets
2. Format questions for multiple-choice evaluation
3. Run tinyGPT on benchmark subsets
4. Compute and interpret accuracy metrics
5. Understand why results are limited but instructive

---

## 🧠 Industry Benchmarks Overview

| Benchmark | Purpose | Size |
|-----------|---------|------|
| **MMLU** | World knowledge across 57 subjects | 15,908 questions |
| **HellaSwag** | Commonsense reasoning / story completion | ~40k examples |
| **TruthfulQA** | Truthfulness, avoiding misconceptions | 817 questions |
| **BBH** | Multi-step reasoning (23 hard tasks) | ~6.5k examples |
| **ToxiGen** | Implicit toxicity detection | ~274k statements |

### Why we still evaluate small models:
- Learn the evaluation process and mechanics
- Build reusable evaluation infrastructure
- Understand what capabilities are missing
- Establish baselines for comparison

---

## 🛠️ Step 1: Setup & Install Dependencies

In [ ]:
# Install required packages
!pip install onnxruntime transformers datasets pandas numpy

In [ ]:
# Import libraries
import onnxruntime as ort
from transformers import AutoTokenizer
from datasets import load_dataset
import pandas as pd
import numpy as np

print(f"✅ ONNX Runtime version: {ort.__version__}")

---

## 📦 Step 2: Load Model and Tokenizer

In [ ]:
# Path to the ONNX model
model_path = "/tmp/tinygpt.onnx"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Create inference session
session = ort.InferenceSession(model_path, providers=["CPUExecutionProvider"])

print("✅ Model and tokenizer loaded successfully!")

---

## 📊 Step 3: Load Benchmark Datasets

We'll load small slices from 2-3 benchmarks for efficiency.
Full evaluations take hours—we'll use 50 examples per benchmark.

In [ ]:
# Load HellaSwag - Commonsense reasoning benchmark
print("📥 Loading HellaSwag...")
hellaswag = load_dataset("Rowan/hellaswag", split="validation")
hellaswag_sample = hellaswag.select(range(50))
print(f"   Loaded {len(hellaswag_sample)} examples")

# Load TruthfulQA - Truthfulness benchmark
print("\n📥 Loading TruthfulQA...")
truthfulqa = load_dataset("truthful_qa", "multiple_choice", split="validation")
truthfulqa_sample = truthfulqa.select(range(50))
print(f"   Loaded {len(truthfulqa_sample)} examples")

# Load a simple QA dataset for additional testing
print("\n📥 Loading simple QA dataset (sciq)...")
sciq = load_dataset("sciq", split="test")
sciq_sample = sciq.select(range(50))
print(f"   Loaded {len(sciq_sample)} examples")

print("\n✅ All datasets loaded!")

---

## 🔤 Step 4: Explore Dataset Formats

Each benchmark has a different structure. Let's examine them.

In [ ]:
# Examine HellaSwag format
print("📋 HellaSwag Example:")
print("=" * 50)
example = hellaswag_sample[0]
print(f"Activity: {example['activity_label']}")
print(f"Context: {example['ctx']}")
print(f"Endings:")
for i, ending in enumerate(example['endings']):
    marker = "✓" if i == int(example['label']) else " "
    print(f"  [{marker}] {chr(65+i)}. {ending[:80]}..." if len(ending) > 80 else f"  [{marker}] {chr(65+i)}. {ending}")
print(f"Correct Answer: {chr(65 + int(example['label']))}")

In [ ]:
# Examine TruthfulQA format
print("\n📋 TruthfulQA Example:")
print("=" * 50)
example = truthfulqa_sample[0]
print(f"Question: {example['question']}")
print(f"Category: {example['category']}")
print(f"Choices:")
choices = example['mc1_targets']['choices']
labels = example['mc1_targets']['labels']
for i, (choice, label) in enumerate(zip(choices, labels)):
    marker = "✓" if label == 1 else " "
    print(f"  [{marker}] {chr(65+i)}. {choice[:80]}..." if len(choice) > 80 else f"  [{marker}] {chr(65+i)}. {choice}")

In [ ]:
# Examine SciQ format
print("\n📋 SciQ Example:")
print("=" * 50)
example = sciq_sample[0]
print(f"Question: {example['question']}")
print(f"Correct Answer: {example['correct_answer']}")
print(f"Distractors: {example['distractor1']}, {example['distractor2']}, {example['distractor3']}")

---

## 📐 Step 5: Define Question Formatters

In [ ]:
def format_hellaswag(example: dict) -> tuple:
    """
    Format a HellaSwag example as a multiple-choice question.
    
    Args:
        example: Dataset example with ctx, endings, and label
    
    Returns:
        Tuple of (prompt, correct_answer_letter)
    """
    context = example["ctx"]
    endings = example["endings"]
    
    prompt = f"Context: {context}\n\n"
    prompt += "Which ending makes the most sense?\n"
    for i, ending in enumerate(endings):
        prompt += f"{chr(65+i)}. {ending}\n"
    prompt += "\nAnswer:"
    
    correct_answer = chr(65 + int(example["label"]))
    return prompt, correct_answer


def format_truthfulqa(example: dict) -> tuple:
    """
    Format a TruthfulQA example as a multiple-choice question.
    
    Args:
        example: Dataset example with question and mc1_targets
    
    Returns:
        Tuple of (prompt, correct_answer_letter)
    """
    question = example["question"]
    choices = example["mc1_targets"]["choices"]
    labels = example["mc1_targets"]["labels"]
    
    prompt = f"Question: {question}\n\n"
    for i, choice in enumerate(choices):
        prompt += f"{chr(65+i)}. {choice}\n"
    prompt += "\nAnswer:"
    
    # Find the correct answer (label == 1)
    correct_idx = labels.index(1) if 1 in labels else 0
    correct_answer = chr(65 + correct_idx)
    return prompt, correct_answer


def format_sciq(example: dict) -> tuple:
    """
    Format a SciQ example as a multiple-choice question.
    
    Args:
        example: Dataset example with question, correct_answer, and distractors
    
    Returns:
        Tuple of (prompt, correct_answer_letter)
    """
    question = example["question"]
    correct = example["correct_answer"]
    
    # Combine correct answer with distractors and shuffle
    options = [
        correct,
        example["distractor1"],
        example["distractor2"],
        example["distractor3"]
    ]
    
    # Use consistent shuffling based on question hash for reproducibility
    np.random.seed(hash(question) % 2**32)
    indices = np.random.permutation(4)
    shuffled_options = [options[i] for i in indices]
    correct_idx = list(indices).index(0)  # Original correct was at index 0
    
    prompt = f"Question: {question}\n\n"
    for i, option in enumerate(shuffled_options):
        prompt += f"{chr(65+i)}. {option}\n"
    prompt += "\nAnswer:"
    
    correct_answer = chr(65 + correct_idx)
    return prompt, correct_answer

### Test the formatters

In [ ]:
# Test HellaSwag formatter
prompt, answer = format_hellaswag(hellaswag_sample[0])
print("📋 Formatted HellaSwag Question:")
print("=" * 50)
print(prompt)
print(f"\n(Correct: {answer})")

---

## 🎯 Step 6: Define MCQ Evaluation Function

In [ ]:
def evaluate_tinygpt_on_mcq(
    dataset,
    format_fn,
    session: ort.InferenceSession,
    tokenizer,
    benchmark_name: str
) -> pd.DataFrame:
    """
    Evaluate tinyGPT on a multiple-choice benchmark.
    
    Args:
        dataset: Hugging Face dataset with examples
        format_fn: Function to format examples as (prompt, correct_answer)
        session: ONNX Runtime inference session
        tokenizer: Hugging Face tokenizer
        benchmark_name: Name of the benchmark for logging
    
    Returns:
        DataFrame with evaluation results
    """
    results = []
    correct_count = 0
    
    print(f"\n🔄 Evaluating on {benchmark_name}...")
    
    for i, example in enumerate(dataset):
        # Format the question
        prompt, correct_answer = format_fn(example)
        
        # Tokenize and run inference
        inputs = tokenizer(prompt, return_tensors="np")
        outputs = session.run(None, {"input_ids": inputs["input_ids"]})
        
        # Get logits for the next token
        next_token_logits = outputs[0][0, -1, :]
        
        # Get token IDs for A, B, C, D
        option_tokens = {
            'A': tokenizer.encode(' A', add_special_tokens=False)[-1],
            'B': tokenizer.encode(' B', add_special_tokens=False)[-1],
            'C': tokenizer.encode(' C', add_special_tokens=False)[-1],
            'D': tokenizer.encode(' D', add_special_tokens=False)[-1],
        }
        
        # Find which option has highest probability
        option_logits = {opt: next_token_logits[tid] for opt, tid in option_tokens.items()}
        model_answer = max(option_logits, key=option_logits.get)
        
        # Check if correct
        is_correct = model_answer == correct_answer
        if is_correct:
            correct_count += 1
        
        results.append({
            "benchmark": benchmark_name,
            "question_id": i,
            "correct_answer": correct_answer,
            "model_answer": model_answer,
            "is_correct": is_correct,
            "confidence": float(option_logits[model_answer])
        })
        
        # Progress update
        if (i + 1) % 10 == 0:
            print(f"   Processed {i + 1}/{len(dataset)}...")
    
    accuracy = correct_count / len(dataset) * 100
    print(f"\n✅ {benchmark_name}: {accuracy:.1f}% accuracy ({correct_count}/{len(dataset)})")
    
    return pd.DataFrame(results)

---

## 🚀 Step 7: Run Evaluation on All Benchmarks

In [ ]:
# Run evaluation on HellaSwag
hellaswag_results = evaluate_tinygpt_on_mcq(
    hellaswag_sample,
    format_hellaswag,
    session,
    tokenizer,
    "HellaSwag"
)

In [ ]:
# Run evaluation on TruthfulQA
truthfulqa_results = evaluate_tinygpt_on_mcq(
    truthfulqa_sample,
    format_truthfulqa,
    session,
    tokenizer,
    "TruthfulQA"
)

In [ ]:
# Run evaluation on SciQ
sciq_results = evaluate_tinygpt_on_mcq(
    sciq_sample,
    format_sciq,
    session,
    tokenizer,
    "SciQ"
)

---

## 📊 Step 8: Combine and Analyze Results

In [ ]:
# Combine all results
all_results = pd.concat([
    hellaswag_results,
    truthfulqa_results,
    sciq_results
], ignore_index=True)

# Summary by benchmark
print("\n" + "=" * 60)
print("📊 BENCHMARK RESULTS SUMMARY")
print("=" * 60)

summary = all_results.groupby("benchmark").agg({
    "is_correct": ["sum", "count", "mean"]
}).round(3)

summary.columns = ["Correct", "Total", "Accuracy"]
summary["Accuracy"] = (summary["Accuracy"] * 100).round(1).astype(str) + "%"

print(summary)

# Random baseline comparison
print("\n📈 Comparison to Random Baseline (25% for 4-choice):")
for benchmark in summary.index:
    acc = float(summary.loc[benchmark, "Accuracy"].strip("%"))
    diff = acc - 25
    status = "↑" if diff > 0 else "↓" if diff < 0 else "="
    print(f"   {benchmark}: {acc}% ({status} {abs(diff):.1f}% from random)")

In [ ]:
# Show some example predictions
print("\n📋 Sample Predictions (first 5 per benchmark):")
print("=" * 60)

for benchmark in ["HellaSwag", "TruthfulQA", "SciQ"]:
    print(f"\n{benchmark}:")
    subset = all_results[all_results["benchmark"] == benchmark].head(5)
    for _, row in subset.iterrows():
        status = "✓" if row["is_correct"] else "✗"
        print(f"  [{status}] Q{row['question_id']}: Model={row['model_answer']}, Correct={row['correct_answer']}")

---

## 🤔 Why These Results Are Limited But Instructive

### What we learned:

1. **Evaluation mechanics work** — Our pipeline successfully loads data, formats questions, and computes accuracy
2. **Small models struggle** — Results near random (25%) indicate tinyGPT lacks the capabilities these benchmarks test
3. **Infrastructure is reusable** — The same code works with any ONNX model

### Why results are limited:

1. **Model size** — tinyGPT doesn't have enough parameters to store world knowledge
2. **Training data** — May not have seen similar content during training
3. **Sample size** — 50 examples has high variance; full benchmarks use thousands
4. **Prompt format** — Different prompts can significantly change results

### The key insight:

> **Understanding how to run benchmarks is separate from getting good results.**
> 
> You now have the skills to evaluate any model on industry benchmarks!

---

## 📝 Mini-Project: Design Your Own 10-Question Benchmark

Create a small benchmark to test a specific capability.

### Example: Basic Arithmetic Benchmark

In [ ]:
# Example custom benchmark
my_benchmark = [
    {"question": "What is 2 + 2?", "options": ["3", "4", "5", "6"], "correct": 1},
    {"question": "What is 5 - 3?", "options": ["1", "2", "3", "4"], "correct": 1},
    {"question": "What is 3 × 4?", "options": ["7", "10", "12", "15"], "correct": 2},
    {"question": "What is 10 ÷ 2?", "options": ["3", "4", "5", "6"], "correct": 2},
    {"question": "What is 7 + 8?", "options": ["13", "14", "15", "16"], "correct": 2},
    {"question": "What is 20 - 7?", "options": ["11", "12", "13", "14"], "correct": 2},
    {"question": "What is 6 × 3?", "options": ["15", "16", "18", "21"], "correct": 2},
    {"question": "What is 15 ÷ 3?", "options": ["3", "4", "5", "6"], "correct": 2},
    {"question": "What is 9 + 6?", "options": ["13", "14", "15", "16"], "correct": 2},
    {"question": "What is 100 - 50?", "options": ["25", "40", "50", "75"], "correct": 2},
]

def format_custom_benchmark(example: dict) -> tuple:
    """Format a custom benchmark question."""
    prompt = f"Question: {example['question']}\n\n"
    for i, opt in enumerate(example['options']):
        prompt += f"{chr(65+i)}. {opt}\n"
    prompt += "\nAnswer:"
    return prompt, chr(65 + example['correct'])

# Run evaluation on custom benchmark
print("🧮 Running Custom Arithmetic Benchmark...")
print("=" * 50)

correct = 0
for i, example in enumerate(my_benchmark):
    prompt, correct_answer = format_custom_benchmark(example)
    
    inputs = tokenizer(prompt, return_tensors="np")
    outputs = session.run(None, {"input_ids": inputs["input_ids"]})
    
    next_token_logits = outputs[0][0, -1, :]
    option_tokens = {
        'A': tokenizer.encode(' A', add_special_tokens=False)[-1],
        'B': tokenizer.encode(' B', add_special_tokens=False)[-1],
        'C': tokenizer.encode(' C', add_special_tokens=False)[-1],
        'D': tokenizer.encode(' D', add_special_tokens=False)[-1],
    }
    option_logits = {opt: next_token_logits[tid] for opt, tid in option_tokens.items()}
    model_answer = max(option_logits, key=option_logits.get)
    
    is_correct = model_answer == correct_answer
    if is_correct:
        correct += 1
    
    status = "✓" if is_correct else "✗"
    print(f"[{status}] Q{i+1}: {example['question']} | Model={model_answer}, Correct={correct_answer}")

accuracy = correct / len(my_benchmark) * 100
print(f"\n📊 Custom Benchmark Accuracy: {accuracy:.1f}% ({correct}/{len(my_benchmark)})")

---

## ✅ Knowledge Mastery Checklist

Before moving to Week 5, ensure you can check all boxes:

- [ ] I can load benchmark datasets from Hugging Face
- [ ] I understand the structure of HellaSwag, TruthfulQA, and SciQ
- [ ] I can format MCQ questions for model evaluation
- [ ] I can parse model outputs to extract answer predictions
- [ ] I understand why small models perform near random baseline
- [ ] I created and ran my own custom benchmark

---

**Week 4 Complete!** 🎉

**Next:** *Week 5 — Building a Generic Benchmark Engine*